In [1]:
import os

#os.environ["LD_PRELOAD"] = "/opt/conda/lib/libiomp5.so"
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
#os.environ["MKL_VERBOSE"] = "1"
os.environ["OMP_DISPLAY_ENV"] = "TRUE"
os.environ["KMP_AFFINITY"] = "verbose,granularity=fine,compact,1,0"
os.environ["KMP_BLOCKTIME"] = "1"
#os.environ["OMP_WAIT_POLICY"] = "active"
#os.environ["KMP_LIBRARY"] = "turnaround"



In [2]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler

print(torch.__version__)


OPENMP DISPLAY ENVIRONMENT BEGIN
  _OPENMP = '201511'
  OMP_DYNAMIC = 'FALSE'
  OMP_NESTED = 'FALSE'
  OMP_NUM_THREADS = '8'
  OMP_SCHEDULE = 'DYNAMIC'
  OMP_PROC_BIND = 'FALSE'
  OMP_PLACES = ''
  OMP_STACKSIZE = '23000636197201'
  OMP_WAIT_POLICY = 'PASSIVE'
  OMP_THREAD_LIMIT = '4294967295'
  OMP_MAX_ACTIVE_LEVELS = '2147483647'
  OMP_CANCELLATION = 'FALSE'
  OMP_DEFAULT_DEVICE = '0'
  OMP_MAX_TASK_PRIORITY = '0'
OPENMP DISPLAY ENVIRONMENT END

OPENMP DISPLAY ENVIRONMENT BEGIN
   _OPENMP='201611'
  [host] OMP_AFFINITY_FORMAT='OMP: pid %P tid %i thread %n bound to OS proc set {%A}'
  [host] OMP_ALLOCATOR='omp_default_mem_alloc'
  [host] OMP_CANCELLATION='FALSE'
  [host] OMP_DEBUG='disabled'
  [host] OMP_DEFAULT_DEVICE='0'
  [host] OMP_DISPLAY_AFFINITY='FALSE'
  [host] OMP_DISPLAY_ENV='TRUE'
  [host] OMP_DYNAMIC='FALSE'
  [host] OMP_MAX_ACTIVE_LEVELS='1'
  [host] OMP_MAX_TASK_PRIORITY='0'
  [host] OMP_NESTED: deprecated; max-active-levels-var=1
  [host] OMP_NUM_TEAMS='0'
  [host] OMP

2.0.0+cpu


In [3]:
def data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):

    transform = transforms.Compose([transforms.ToTensor()])
    
    if test:
        dataset = datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        return data_loader

    # load the dataset
    train_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    valid_dataset = datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)

In [4]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.fc1 = nn.Sequential(
            nn.Linear(64*7*7, 512),
            nn.Dropout(0.25))
        
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [5]:
#optimizer = torch.optim.Adam
max_lr = 0.00001
grad_clip = 0.1
weight_decay = 0.005

# Device configuration
device = 'cpu'
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate = 0.0001

# CIFAR10 dataset 
train_loader, valid_loader = data_loader(data_dir='./data', batch_size=batch_size)
test_loader = data_loader(data_dir='./data', batch_size=batch_size, test=True)

#model = VGG_test(num_classes).to(device)
model = CNN(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#scheduler = torch.optim.lr_scheduler.OneCycleLR
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)
optimizer = torch.optim.Adam(model.parameters(), max_lr, weight_decay=weight_decay)
#scheduler = scheduler(optimizer, max_lr, epochs=num_epochs, steps_per_epoch=len(train_loader))

for (images, _) in train_loader:
    model(images)

OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659941 thread 1 bound to OS proc set 1
OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659942 thread 2 bound to OS proc set 2
OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659943 thread 3 bound to OS proc set 3
OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659944 thread 4 bound to OS proc set 4
OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659945 thread 5 bound to OS proc set 5
OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659946 thread 6 bound to OS proc set 6
OMP: Info #255: KMP_AFFINITY: pid 659885 tid 659947 thread 7 bound to OS proc set 7


In [6]:
%%time
# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {:.2f} %'.format(5000, 100 * correct / total)) 

Epoch [1/5], Step [844/844], Loss: 0.4573
Accuracy of the network on the 5000 validation images: 83.92 %
Epoch [2/5], Step [844/844], Loss: 0.3681
Accuracy of the network on the 5000 validation images: 86.35 %
Epoch [3/5], Step [844/844], Loss: 0.3125
Accuracy of the network on the 5000 validation images: 87.37 %
Epoch [4/5], Step [844/844], Loss: 0.3913
Accuracy of the network on the 5000 validation images: 88.18 %
Epoch [5/5], Step [844/844], Loss: 0.3661
Accuracy of the network on the 5000 validation images: 88.45 %
CPU times: user 7min 23s, sys: 1min 1s, total: 8min 25s
Wall time: 1min 18s


In [7]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))   

Accuracy of the network on the 10000 test images: 87.52 %
